增加priority欄位到資料表

In [1]:
# ! pip install selenium

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 968 kB 312 kB/s eta 0:00:01
     |████████████████████████████████| 359 kB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 8.1 MB/s  eta 0:00:01


In [1]:
from bs4 import BeautifulSoup
import requests
import urllib3
from pymongo import MongoClient
import re
from multiprocessing import Pool
import traceback
from selenium import webdriver
import time
import pandas as pd
import os

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
! nvidia-smi

Sun May  8 03:03:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 38%   48C    P8    40W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 32%   

# Soup

In [4]:
def handle_error(e):
    #處理 child process 的錯誤，不然 code 寫錯時，不會回報任何錯誤
    traceback.print_exception(type(e), e, e.__traceback__)

In [5]:
def patent_soup(patent_num):
    urllib3.disable_warnings() #禁用各種警告
    response = requests.get('https://patents.google.com/patent/' + str(patent_num),verify = False)
    response.encoding='utf-8'
    if response.status_code != 200:
        pass
    else:
        soup =  BeautifulSoup(response.text, 'html.parser') 
        try:
            appDate = soup.find('time', itemprop="filingDate").string.replace('-', '')
            
            # Classifications
            try:
                classifications = soup.find_all('span', itemprop="Code")
                classification =[]
                for i in range(0,len(classifications)):
                    try:
                        if len(classifications[i].string)>len(classifications[i+1].string):
                            classification.append(classifications[i].string)
                        elif i == len(classifications)-2:
                            classification.append(classifications[i+1].string) 
                    except:
                        pass
                if classification[0] == '':
                    classification = None
            except:
                classification = None
        except:
            return None
        
    return appDate, classification

In [6]:
# 取得html
def soup(url):
    urllib3.disable_warnings() #禁用各種警告
    response = requests.get(url,verify = False)
    response.encoding='utf-8'
    if response.status_code != 200:
        pass
    else:
        soup =  BeautifulSoup(response.text, 'html.parser') 
        try:
            patent_num = soup.find('dd',itemprop='publicationNumber').string
            appDate = soup.find('time', itemprop="filingDate").string.replace('-', '')
            pubDate = soup.find('time', itemprop="publicationDate").string.replace('-', '')
            abstract = soup.find('div',{"class":"abstract"}).string
            title = re.sub('\s','',soup.find('span',itemprop="title").string)
            
            #current assignee
            try:
                current = soup.find_all('dd' ,itemprop="assigneeCurrent")
                current_assignee = []
                for i in current:
                    i = re.sub('\n','',i.string)
                    i = ' '.join(i.split())
                    current_assignee.append(i)
            except:
                current_assignee = None
            
            #original assignee
            try:
                original = soup.find_all('dd' ,itemprop="assigneeOriginal")
                original_assignee = []
                for i in original:
                    original_assignee.append(i.string)
            except:
                original_assignee = None
            
            #graDate
            Date = soup.find_all('tr',itemprop="legalEvents")
            graDate = 'N/A'
            for i in Date:
#                 if re.search('.*Grant.*',i.text) != None or re.search('.*grant.*',i.text) != None:
#                     graDate =re.sub('\s','',i.text)[:10].replace('-', '')
#                 else:
#                     graDate = graDate
                if i.find('td', itemprop='title').text == 'Information on status: patent grant':
                    graDate =re.sub('\s','',i.text)[:10].replace('-', '')
            
            # inventorName
            try:
                inventorNames = soup.find_all('dd',itemprop="inventor")
                inventorName = []
                for i in inventorNames:
                    inventorName.append(i.string)
            except:
                inventorName = None

            # description
#             try:
#                 description = soup.find_all('p')
#                 description_list = []
#                 for i in description:
#                     description_list.append(re.sub('\s','',i.text))
#                 description = ("".join(description_list)) 
#             except:
#                 description = None
            if graDate != 'N/A':
                try:
                    descriptions = soup.find_all('div', class_="description-paragraph") #CN
                    description = ''
                    for i in descriptions:
                        description += i.text
                except:
                    description = None
            else:
                try:
                    descriptions = soup.find_all('div', class_="description-line")
                    description = ''
                    if len(descriptions) != 0:
                        for i in descriptions:
                            description += i.text
                    else:
                        descriptions = soup.find_all('div', class_="description-paragraph") #CN
                        for i in descriptions:
                            description += i.text
                except:
                    description = None
                                
            # detailed_description
#             try:
#                 detailed_description_list = soup.find_all('p')
#                 detailed_description_start = 0
#                 detailed_description = ''
                            
#                 for num,i in enumerate(detailed_description_list):
#                     if re.sub('\s','',i.text) == '具体实施方式':
#                         detailed_description_start = num
#                 if detailed_description_start > 0:
#                     for i in range(detailed_description_start,len(detailed_description_list)):
#                         detailed_description += re.sub('\s','',detailed_description_list[i].text)
#                     detailed_description = "".join(detailed_description)
#                 else:
#                     detailed_description = None
#             except:
#                 detailed_description = None
                
            # claim
            try:
                claims = soup.find_all('div',{'class':'claim-text'})
                claim = []
                j = 0
                for i in claims:
                    if re.sub('\s','',i.text) != '':
                        claim.append(re.sub('\s','',i.text))
            #分類
                for i in range(1,len(claim)-1):
                    try:
                        int(claim[i][0])
                        j = i
                    except:
                        a = str(claim[j])+str(claim[i])
                        claim[j] = a
                        claim[i] = '' 
                for i in claim:
                    if '' in claim:
                        claim.remove('')
            except:
                claim = None

            # patentCited    
            try:
                patentCiteds = soup.find_all('tr', itemprop="backwardReferences")
                patentCited_list = []
                patentCited_assignee_list = []
                patentCited = []
                for i in patentCiteds:
                    patentCited_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    patentCited_list.append(re.sub('\s','',i.text)[:e])
                if len(patentCited_list) != 0 :
                    for j in patentCited_list:
                        pclist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        pclist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                pclist['patentNumber'] = j[2:]
                                pclist['kindcode'] = None
                            except:
                                pclist['patentNumber'] = j[2:-2]
                                pclist['kindcode'] = j[-1]
                        except:
                            pclist['patentNumber'] = j[2:-2]
                            pclist['kindcode'] = j[-2:]
                        try:                   
                            cited_appDate, cited_classification = patent_soup(j)
                            pclist['appDate'] = cited_appDate
                            pclist['classification'] = cited_classification
                        except:
                            pclist['appDate'] = None
                            pclist['classification'] = None
                        patentCited.append(pclist)
                    for i in range(len(patentCited)):
                        try:
                            patentCited[i]['assignee'] = patentCited_assignee_list[i]
                        except:
                            patentCited[i]['assignee'] = None
                else:
                    patentCited = None
                    
            except:
                patentCited = None
                
            # Family To Family Citing      
            try:
                family_to_family_citings = soup.find_all('tr', itemprop="backwardReferencesFamily")
                family_to_family_citing_list = []
                family_to_family_citing_assignee_list = []
                family_to_family_citing = []
                for i in family_to_family_citings:
                    family_to_family_citing_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    family_to_family_citing_list.append(re.sub('\s','',i.text)[:e])
                    
                if len(family_to_family_citing_list) != 0 :
                    for j in family_to_family_citing_list:
                        clist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        clist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                clist['patentNumber'] = j[2:]
                                clist['kindcode'] = None
                            except:
                                clist['patentNumber'] = j[2:-2]
                                clist['kindcode'] = j[-1]
                        except:
                            clist['patentNumber'] = j[2:-2]
                            clist['kindcode'] = j[-2:]
                        try:                   
                            citing_appDate, citing_classification = patent_soup(j)
                            clist['appDate'] = citing_appDate
                            clist['classification'] = citing_classification
                        except:
                            clist['appDate'] = None
                            clist['classification'] = None
                        family_to_family_citing.append(clist) 
                    for i in range(len(family_to_family_citing)):
                        try:
                            family_to_family_citing[i]['assignee'] = family_to_family_citing_assignee_list[i]
                        except:
                            family_to_family_citing[i]['assignee'] = None
                else:
                    family_to_family_citing = None
            except:
                family_to_family_citing = None
                
            # ＣountryStatus
            try:
                status =  soup.find_all('tr', itemprop="countryStatus")
                countrystatus = []
                for i in status:
                    countrycode = i.find('span', itemprop="countryCode")
                    try:
                        countrystatus.index(countrycode.string)
                    except:
                        countrystatus.append(countrycode.string)
            except:
                countrystatus = None
            
            # Cited       
            try:
                citeds_1 = soup.find_all('tr' ,itemprop="forwardReferencesOrig" )
                #citeds_2 = soup.find_all('tr', itemprop="forwardReferencesFamily")
                cited_list = []
                cited_assignee_list = []
                cited = []
                for i in citeds_1:
                    cited_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    cited_list.append(re.sub('\s','',i.text)[:e])
#                 for i in citeds_2:
#                     e = re.sub('\s','',i.text).find("(")
#                     cited_list.append(re.sub('\s','',i.text)[:e])
                    
                if len(cited_list) != 0 :
                    for j in cited_list:
                        clist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        clist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                clist['patentNumber'] = j[2:]
                                clist['kindcode'] = None
                            except:
                                clist['patentNumber'] = j[2:-2]
                                clist['kindcode'] = j[-1]
                        except:
                            clist['patentNumber'] = j[2:-2]
                            clist['kindcode'] = j[-2:]
                        try:                   
                            cited_appDate, cited_classification = patent_soup(j)
                            clist['appDate'] = cited_appDate
                            clist['classification'] = cited_classification
                        except:
                            clist['appDate'] = None
                            clist['classification'] = None
                        cited.append(clist)
                    for i in range(len(cited)):
                        try:
                            cited[i]['assignee'] = cited_assignee_list[i]
                        except:
                            cited[i]['assignee'] = None
                
                else:
                    cited = None
            except:
                cited = None
                
            # Family To Family Cited      
            try:
                family_to_family_citeds = soup.find_all('tr', itemprop="forwardReferencesFamily")
                family_to_family_cited_list = []
                family_to_family_cited_assignee_list = []
                family_to_family_cited = []
                for i in family_to_family_citeds:
                    family_to_family_cited_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    family_to_family_cited_list.append(re.sub('\s','',i.text)[:e])
                    
                if len(family_to_family_cited_list) != 0 :
                    for j in family_to_family_cited_list:
                        clist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        clist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                clist['patentNumber'] = j[2:]
                                clist['kindcode'] = None
                            except:
                                clist['patentNumber'] = j[2:-2]
                                clist['kindcode'] = j[-1]
                        except:
                            clist['patentNumber'] = j[2:-2]
                            clist['kindcode'] = j[-2:]
                        try:                   
                            cited_appDate, cited_classification = patent_soup(j)
                            clist['appDate'] = cited_appDate
                            clist['classification'] = cited_classification
                        except:
                            clist['appDate'] = None
                            clist['classification'] = None
                        family_to_family_cited.append(clist) 
                    for i in range(len(family_to_family_cited)):
                        try:
                            family_to_family_cited[i]['assignee'] = family_to_family_cited_assignee_list[i]
                        except:
                            family_to_family_cited[i]['assignee'] = None
                else:
                    family_to_family_cited = None
            except:
                family_to_family_cited = None

            # Classifications
            try:
                classifications = soup.find_all('span', itemprop="Code")
                classification =[]
                for i in range(0,len(classifications)):
                    try:
                        if len(classifications[i].string)>len(classifications[i+1].string):
                            classification.append(classifications[i].string)
                        elif i == len(classifications)-2:
                            classification.append(classifications[i+1].string) 
                    except:
                        pass
                if classification[0] == '':
                    classification = None
            except:
                classification = None
                
            # Similar Documents
            try:
                similar_docs = soup.find_all('tr', itemprop="similarDocuments")
                similarDocuments = []
                similar_doc =[]
                doc_title = []
                doc_publication = []
                doc_time = []

                for doc in similar_docs:
                    publication = doc.find_all('span')
                    time = doc.find_all('time')
                    titles = doc.find_all('td', itemprop="title")

                    for j1 in publication:
                        if len(j1.text.strip())!=2:
                            doc_publication.append(j1.text.strip())
                    for j2 in time:
                        doc_time.append(j2.text.strip())
                    for j3 in titles:
                        doc_title.append(j3.text.strip())
                similarDocuments = []
                for i in range(len(doc_publication)):
                    doc={
                        'publication' : doc_publication[i],
                        'time' : doc_time[i],
                        'title' : doc_title[i]
                        }
                    similarDocuments.append(doc)
            except:
                similarDocuments = None
                

            data = {
            'patentNumber':patent_num, 
            'inventorName':inventorName, 
            'current_assignee':current_assignee, 
            'original_assignee':original_assignee,
            'appDate':appDate, # application Date
            'pubDate':pubDate , # public Date
            'graDate':graDate, #
            'classification':classification, # 類別
            'brief':abstract, # 摘要
            'title':title, # 標題
            'description':description, 
#             'detailed_description':detailed_description,
            'claim':claim,
            'countrystatus':countrystatus,
            'patentCitation':patentCited,
            'family_to_family_citing':family_to_family_citing,
            'cited':cited,
            'family_to_family_cited':family_to_family_cited,
            'similarDocuments':similarDocuments
            }
            return data
        except:
            #print("except==",url)
            #error_url.append(url)
            #pass
            
            return None

In [29]:
# 取得html
def soup(url):
    urllib3.disable_warnings() #禁用各種警告
    response = requests.get(url,verify = False)
    response.encoding='utf-8'
    if response.status_code != 200:
        pass
    else:
        soup =  BeautifulSoup(response.text, 'html.parser') 
        try:
            patent_num = soup.find('dd',itemprop='publicationNumber').string
            appDate = soup.find('time', itemprop="filingDate").string.replace('-', '')
            pubDate = soup.find('time', itemprop="publicationDate").string.replace('-', '')
            abstract = soup.find('div',{"class":"abstract"}).string
            title = re.sub('\s','',soup.find('span',itemprop="title").string)
            
            #current assignee
            try:
                priority = soup.find_all('span', itemprop="applicationNumber")[1].text
                # priority_ls = []
                # priority_ls.append(priority)
            except:
                priority_ls = None
            
                

            data = [{'patentNumber':patent_num},{'priority':priority}]
            # data = {'priority':priority_ls}
            return data
        except:
            #print("except==",url)
            #error_url.append(url)
            #pass
            
            return None

In [8]:
#搜尋google patent
def search_google_patent(keyword='', assignee=[], s_date='', e_date=''):
    assignees = ''
    for i in assignee:
        for j in i.split():
            assignees += str(j)
            assignees += '+'
        assignees += ','
    url = 'https://patents.google.com/?q=' + keyword + '&assignee=' + assignees + '&country=US&before=filing:' + e_date + '&after=filing:' + s_date + '&language=ENGLISH&type=PATENT'
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-infobars')
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome(chrome_options=options, executable_path='./chromedriver')
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    driver.close()
    soup = BeautifulSoup(html, "html.parser")
    
    csv_url = 'https://patents.google.com/' + soup.find('a', class_='style-scope search-results')['href']
    urllib.request.urlretrieve(csv_url, "US_patent.csv")

# MongoDB

In [9]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm

In [10]:
urllib3.disable_warnings() #禁用各種警告
response = requests.get("https://patents.google.com/patent/CN103947135B/zh",verify = False)
response.encoding='utf-8'
if response.status_code != 200:
    pass
else:
    soup =  BeautifulSoup(response.text, 'html.parser') 

In [11]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<title>CN103947135B - 用于无线网络中的单独和组寻呼的系统、设备和方法 
        - Google Patents</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta charset="utf-8"/>
<meta content="origin-when-crossorigin" name="referrer"/>
<link href="https://patents.google.com/patent/CN103947135B/zh" rel="canonical"/>
<meta content="
     本文描述了用于经无线通信网络单独和组寻呼无线装置的方法和系统。所述方法能够包括设置用于无线装置的寻呼周期和寻呼偏移。使用装置组标识符，能够将无线装置指派到装置组。在寻呼周期的一部分期间，无线装置能够醒来以检查经无线通信网络接收的寻呼事件。又一操作能够是在寻呼周期的该部分期间在无线装置接收对于装置组的组寻呼事件。 
     
   " name="description"/>
<meta content="patent" name="DC.type"/>
<meta content="用于无线网络中的单独和组寻呼的系统、设备和方法 
       " name="DC.title"/>
<meta content="2012-04-17" name="DC.date" scheme="dateSubmitted"/>
<meta content="
     本文描述了用于经无线通信网络单独和组寻呼无线装置的方法和系统。所述方法能够包括设置用于无线装置的寻呼周期和寻呼偏移。使用装置组标识符，能够将无线装置指派到装置组。在寻呼周期的一部分期间，无线装置能够醒来以检查经无线通信网络接收的寻呼事件。又一操作能够是在寻呼周期的该部分期间在无线装置接收对于装置组的组寻呼事件。 
     
   " name="DC.description"/>
<meta content="CN:201280059351.9A" name="citati

In [12]:
descriptions = soup.find_all('span', itemprop="applicationNumber")[1].text

In [13]:
descriptions

'US201161556109P'

In [14]:
descriptions = soup.find('span', itemprop="applicationNumber").text

In [15]:
descriptions[1].text

AttributeError: 'str' object has no attribute 'text'

In [16]:
len(descriptions)

17

## DE

In [32]:
# MongoDB 連線
def update_data(data):
    mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    conn_db_collection = db.patent_de
    
    # conn_db_collection.insert_one(data) #insert data
    # update data
    conn_db_collection.update_one(data[0],{"$set":data[1]}) 

In [ ]:
#爬蟲並新增資料
data_DE = pd.read_csv('../years/DE/DE_all.csv', encoding= 'unicode_escape')
# urls = list(data.iloc[:, 0])
# del urls[0]

de_error_ls = []
for i in tqdm(range(0,len(data_DE))): 
    try:
        data_s = soup(data_DE["result link"][i])
        update_data(data_s)
    except:
        print("error===",i)
        de_error_ls.append(i)
        pass

 24%|██▍       | 449/1833 [03:48<09:27,  2.44it/s]  

In [ ]:
soup(data_DE["result link"][0])[1]

In [31]:
soup(data_DE["result link"][0])

[{'patentNumber': 'DE112012005419B4'}, {'priority': 'CN201110456813.8A'}]

## CN

In [21]:
# MongoDB 連線
def update_data(data):
    mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    conn_db_collection = db.patent_cn
    
    # conn_db_collection.insert_one(data) #insert data
    # update data
    conn_db_collection.update_one(data[0],{"$set":data[1]}) 

In [22]:
#爬蟲並新增資料
data_CN = pd.read_csv('../years/CN/CN_all.csv')
# urls = list(data.iloc[:, 0])
# del urls[0]

cn_error_ls = []
for i in tqdm(range(6400,len(data_CN))): #len(data_CN)
    try:
        data_s = soup(data_CN["result link"][i])
        update_data(data_s)
    except:
        print("error===",i,"id==",data_CN["result link"][i])
        cn_error_ls.append(data_CN["result link"][i])
        pass

 16%|█▌        | 6371/40913 [00:27<02:30, 228.89it/s]

error=== 12726 id== https://patents.google.com/patent/CN106227059A/zh


 21%|██        | 8612/40913 [00:37<02:20, 230.14it/s]

error=== 14967 id== https://patents.google.com/patent/CN206945686U/zh


 23%|██▎       | 9217/40913 [00:40<02:18, 228.48it/s]


KeyboardInterrupt: 

In [12]:
# data_CN = pd.read_csv('../years/CN/CN_all.csv')
# len(data_CN)

47313

## US

In [ ]:
# MongoDB 連線
def update_data(data):
    mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    conn_db_collection = db.patent_us
    
    # conn_db_collection.insert_one(data) #insert data
    # update data
    conn_db_collection.update_one(data[0],{"$set":data[1]}) 

In [ ]:
#爬蟲並新增資料
data_US = pd.read_csv('../years/US/US_all.csv')

us_error_ls = []
for i in tqdm(range(0,len(data_US))):
    try:
        data_s = soup(data_US["result link"][i])
        update_data(data_s)
    except:
        print("error===",i,"id==",data_US["result link"][i])
        us_error_ls.append(data_US["result link"][i])
        pass